In [3]:
import json
import pandas as pd
from pandas import json_normalize
#from pandas.json_normalize import json_normalize

import os
import configparser
import datetime as dt
from datetime import datetime

# Method to read config file settings
def read_config(Config_File):
    config = configparser.ConfigParser()
    config.read(Config_File)
    return config

configurations = read_config("configurations.ini")

In [4]:
for each_section in configurations.sections():
    print("Section: {}".format(each_section))
    for (each_key, each_val) in configurations.items(each_section):
        print("key {} : Value {}".format(each_key,each_val))

Section: common_settings
key gtfs_history_data_root : Value C:\Users\AYAB\OneDrive - Drexel University\drexel\DSCI 591\detour project\data\GTFS-RT-historical-2022


In [5]:
print(configurations['common_settings']['gtfs_history_data_root'])

C:\Users\AYAB\OneDrive - Drexel University\drexel\DSCI 591\detour project\data\GTFS-RT-historical-2022


In [6]:
#load 1 file in panda frame
month = '10'
day = '3'
hour = '09'
minute = '00'

file = configurations['common_settings']['gtfs_history_data_root'] + "\\" + month + '\\' + day + '\\' + hour + '\\' + minute + '\\' + 'feed.json'

with open(file) as json_file:
    data = json.load(json_file)
    #print(data['entity'])

gtfs_rt_entities = json_normalize(data['entity'])

# initial data cleaning, remove the vehicle that have not trip/route data
gtfs_rt_entities = gtfs_rt_entities.dropna(subset = ['vehicle.trip.tripId'])
#gtfs_rt_entities.head(5)

gtfs_rt_entities.to_csv('data\out.csv')

gtfs_rt_entities.head(5)

,id,vehicle.position.latitude,vehicle.position.longitude,vehicle.position.speed,vehicle.timestamp,vehicle.vehicle.id,vehicle.vehicle.label,vehicle.trip.tripId,vehicle.trip.startDate,vehicle.trip.routeId,vehicle.position.bearing,vehicle.currentStopSequence,vehicle.currentStatus,vehicle.stopId
1,8093,40.113320,-75.345474,1.403706,1664787605,8093,8093,50172,20221003,93,116.468094,1.0,STOPPED_AT,1649
5,8097,40.099823,-75.464500,18.158764,1664787606,8097,8097,49433,20221003,139,79.336075,48.0,IN_TRANSIT_TO,28972
12,3205,40.124863,-75.338715,5.145431,1664787605,3205,3205,50886,20221003,96,48.636463,9.0,STOPPED_AT,27398
13,3202,40.067900,-75.337975,16.263315,1664787605,3202,3202,47506,20221003,124,272.992030,13.0,IN_TRANSIT_TO,30333
16,3217,40.143420,-75.391365,15.011603,1664787603,3217,3217,50188,20221003,93,135.278090,58.0,STOPPED_AT,30432


In [40]:
#TODO: create a parameter for file output

def load_gtfs_rt(start_date_time_str, stop_date_time_str):
    """ load GTFS RT file into a panda DF and store result to csv file"""
    start_date_time = datetime.strptime(start_date_time_str, '%m/%d/%Y %H:%M')
    stop_date_time = datetime.strptime(stop_date_time_str, '%m/%d/%Y %H:%M')
    
    # delta time
    delta = dt.timedelta(minutes=1)
    
    #initialize the file with header from dataframe
    with open('data\out.csv', 'w') as f:
        f.write('id,vehicle.position.latitude,vehicle.position.longitude,vehicle.position.speed,vehicle.timestamp,' + \
            'vehicle.vehicle.id,vehicle.vehicle.label,vehicle.trip.tripId,vehicle.trip.startDate,vehicle.trip.routeId,' +  
            'vehicle.position.bearing,vehicle.currentStopSequence,vehicle.currentStatus,vehicle.stopId')
        f.write("\n") # need to add the carriage return this way

    # iterate over range of date / time
    while (start_date_time <= stop_date_time):
        # print(start_date_time, end="\n")
        #generate the file name to open
        file = configurations['common_settings']['gtfs_history_data_root'] + "\\" \
            + start_date_time.strftime("%m") + '\\' \
            + start_date_time.strftime("%#d") + '\\' \
            + start_date_time.strftime("%H") + '\\' \
            + start_date_time.strftime("%M") + '\\' + 'feed.json'

        #load the file name into mem
        #TODO: if I don't find a file fail gracefully. print out the file not found.
        with open(file) as json_file:
            data = json.load(json_file)

        # load json into flat dictionary
        gtfs_rt_entities = json_normalize(data['entity'])

        # initial data cleaning, remove the vehicle that have not trip/route data
        gtfs_rt_entities = gtfs_rt_entities.dropna(subset = ['vehicle.trip.tripId'])


        gtfs_rt_entities.to_csv('data\out.csv', mode='a', index=False, header=False)
        # increemnt datetime by 1 minute
        start_date_time += delta


In [42]:
# start_Date_time_str = "10/3/2022 9:00"

# start_Date_time = datetime.strptime(start_Date_time_str, '%m/%d/%Y %H:%M')

load_gtfs_rt("10/03/2022 9:00","10/03/2022 19:59")

# print('shape {}'.format(gtfs_rt_entities.shape))

OSError: [Errno 22] Invalid argument: 'C:\\Users\\AYAB\\OneDrive - Drexel University\\drexel\\DSCI 591\\detour project\\data\\GTFS-RT-historical-2022\\10\\3\\13\\54\\feed.json'

In [36]:
df = pd.read_csv('data\out.csv')

df.head(5)

,id,vehicle.position.latitude,vehicle.position.longitude,vehicle.position.speed,vehicle.timestamp,vehicle.vehicle.id,vehicle.vehicle.label,vehicle.trip.tripId,vehicle.trip.startDate,vehicle.trip.routeId,...,1664787605,8093,8093.1,50172,20221003,93,116.468094,1.0,STOPPED_AT,1649
0,8097,40.099823,-75.464500,18.158764,1664787606,8097,8097,49433,20221003,139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3205,40.124863,-75.338715,5.145431,1664787605,3205,3205,50886,20221003,96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3202,40.067900,-75.337975,16.263315,1664787605,3202,3202,47506,20221003,124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3217,40.143420,-75.391365,15.011603,1664787603,3217,3217,50188,20221003,93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3233,39.978080,-75.154180,12.190781,1664787604,3233,3233,61589,20221003,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
